In [ ]:
# Installing the 'praw' and 'ratelimiter' packages using the pip package manager

!pip install praw ratelimiter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
import praw
import time
from ratelimiter import RateLimiter

In [ ]:
import csv


In [ ]:
# Setting up the Reddit API client
reddit = praw.Reddit(
    client_id='gnb67sVnHPSBfy-77dLXtA',
    client_secret='UM3yQxj_0HdDHmGSWt2APmJXoBUoHA',
    user_agent='script:MyStudyScraperApp:v1.0 (by /u/Suspicious_Video_367)'
)

# Defining the subreddits and search terms
subreddits = ['rheumatoidarthritis']
search_terms = ['Methotrexate', 'mtx','Meloxicam', 'Sulfasalazine', 'Hydroxychloroquine', 'Leflunomide','Cyclosporine']

def process_submission(submission, csv_writer):
    # Saving submission details
    csv_writer.writerow([submission.title, 'post', submission.selftext, submission.score, submission.id, submission.url, submission.created])

    # Expanding the comment tree of the submission
    submission.comments.replace_more(limit=0)
    comments = submission.comments.list()

    # Saving comments
    for comment in comments:
        # Check if the comment has a body
        if comment.body:
            csv_writer.writerow([submission.title, 'comment', comment.body, comment.score, comment.id, '', comment.created])

# Main loop for scraping
with open('scraped_data.csv', mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['title', 'type', 'text', 'score', 'id', 'url', 'created'])  # Header row

    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)

        for term in search_terms:
            query = term

            try:
                submissions = subreddit.search(query, sort='comments', time_filter='year', limit=100)

                submissions_list = list(submissions)
                if not submissions_list:
                    print(f"No results for '{term}' in '{subreddit_name}'")
                    continue

                for submission in submissions_list:
                    process_submission(submission, csv_writer)

                time.sleep(2)  # Handling rate limits

            except praw.exceptions.APIException as e:
                print(f"API Exception: {e}")
                time.sleep(10)
            except praw.exceptions.ClientException as e:
                print(f"Client Exception: {e}")
                time.sleep(10)
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                time.sleep(10)